In [77]:
import pandas as pd
from binance.client import Client
from Binance_api import api_key, secret_key
from datetime import datetime
from datetime import timedelta
import time
from binance.exceptions import BinanceAPIException
import pandas as pd
import numpy as np
import os
from binance.client import Client
from Binance_api import api_key, secret_key
import time
import plotly.graph_objects as go
import talib as ta
from datetime import datetime
import os
from ta import add_all_ta_features
import warnings
import sklearn
from sklearn.decomposition import PCA
import plotly.express as px
import numpy
import plot_utils 
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator

warnings.filterwarnings('ignore')

client = Client(api_key, secret_key,tld='us')
rsi = 0
bottoms_5 = [9999999,99999999,99999999,99999999,9999999]
rsi_l = [9999999,99999999,99999999,99999999,9999999]

In [78]:
df = pd.read_csv('DOGE_5min_10AUG.csv')

In [79]:
def sma(data, window): 
    return(data.rolling(window = window).mean())

def bollinger_band(data, sma, window, nstd):
    std = data.rolling(window = window).std()
    upper_band = sma + std * nstd
    lower_band = sma - std * nstd
    return upper_band, lower_band

symbols = ['DOGE']

nstd = 3
def rsi(symbol, i):
    if df[f'{symbol}_rsi'].iloc[i] > rsi:
        rsi = df[f'{symbol}_rsi'].iloc[i]
    return rsi

def bottoms(new_bottom,rsi_new):
    is_new = 0
    for a in range(len(bottoms_5)):
        if new_bottom < bottoms_5[a]:
            is_new = 1
    if is_new == 1:
        br = len(bottoms_5) - 1
        while br > 0:
            bottoms_5[br] = bottoms_5[br-1]
            rsi_l[br] = rsi_l[br-1]
            br -= 1
        br[0] = new_bottom
        rsi_l = rsi_new
     
def divergence():
    low_n = 0
    for a in range(len(bottoms)):
        if bottoms[0] < bottoms[a] and rsi_l[0] > rsi_l[a]:
            low_n =+ 1 
    if low_n == 4:
        return True


    



for symbol in symbols:
    df[f'{symbol}_sma'] = sma(df[f'{symbol}-USD_Open'],20)
    df[f'{symbol}_upper_band'], df[f'{symbol}_lower_band'] = bollinger_band(df[f'{symbol}-USD_Open'], df[f'{symbol}_sma'], 20, nstd)        
    df[f'{symbol}_rsi'] = ta.RSI(df[f'{symbol}-USD_Close'])

        
df.dropna(inplace=True)

In [80]:
class TradingEnv:
    def __init__(self, balance_amount, balance_unit, trading_fee_multiplier):
        self.balance_amount = balance_amount
        self.balance_unit = balance_unit
        self.buys = []
        self.sells = []
        self.trading_fee_multiplier = trading_fee_multiplier
        
    def buy(self, symbol, buy_price, time):
        self.balance_amount = (self.balance_amount / buy_price) * self.trading_fee_multiplier
        self.balance_unit = symbol
        self.buys.append([symbol, time, buy_price])
        
    def sell(self, sell_price, time):
        self.balance_amount = self.balance_amount * sell_price * self.trading_fee_multiplier
        self.sells.append( [self.balance_unit, time, sell_price] )
        self.balance_unit = 'USDT'

# VIP level 0, paying fees with BNB = 0.075%

        

In [81]:
env = TradingEnv(balance_amount=100,balance_unit='USDT', trading_fee_multiplier=1) #0.99925

In [82]:
last_buy = 0
for i in range(len(df)):
    if env.balance_unit == 'USDT':
        for symbol in symbols:
            #if df[f'{symbol}-USD_Low'].iloc[i] < df[f'{symbol}_lower_band'].iloc[i]:
                #bottoms(df[f'{symbol}-USD_Low'].iloc[i], df[f'{symbol}_lower_band'].iloc[i])

            if df[f'{symbol}-USD_Low'].iloc[i] < df[f'{symbol}_lower_band'].iloc[i] and df[f'{symbol}_rsi'].iloc[i] < 25: #buy signal
                env.buy(symbol, df[f'{symbol}_lower_band'].iloc[i], df['OpenTime'].iloc[i])
                last_buy = df[f'{symbol}_lower_band'].iloc[i]
                break
    if env.balance_unit != 'USDT':
        if last_buy*0.99 > df[f'{env.balance_unit}-USD_Close'].iloc[i]:
            env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i])
        elif df[f'{env.balance_unit}-USD_High'].iloc[i] > df[f'{env.balance_unit}_upper_band'].iloc[i] and df[f'{symbol}_rsi'].iloc[i] > 65: #sell signal
            env.sell(df[f'{env.balance_unit}_upper_band'].iloc[i], df['OpenTime'].iloc[i])

if env.balance_unit != 'USDT':
    env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['OpenTime'].iloc[-1])

In [83]:
print(f'num buys: {len(env.buys)}')
print(f'num sells: {len(env.sells)}')
print('starting balance: 100 USDT')
print(f'ending balance: {env.balance_amount} {env.balance_unit}')


num buys: 144
num sells: 144
starting balance: 100 USDT
ending balance: 55.14113092657486 USDT


In [84]:
balance = 100
profit_total = 0
for i in range(len(env.buys)):
    buys = env.buys[i]
    sells = env.sells[i]
    profit = ((sells[2]/buys[2])-1)*100
    print(buys)
    print(sells)
    print('P/L = 'f'{profit}')
    balance += balance*(profit/100)
    print(balance)
    print('\n')
    

['DOGE', '2022-08-11 04:45:00', 0.06998606575088841]
['DOGE', '2022-08-11 06:45:00', 0.07059856142612005]
P/L = 0.8751680333222156
100.87516803332221


['DOGE', '2022-08-12 10:10:00', 0.07013367190619069]
['DOGE', '2022-08-12 19:25:00', 0.07167562865604567]
P/L = 2.198596919205187
103.09300636994588


['DOGE', '2022-08-16 09:10:00', 0.07558763015785397]
['DOGE', '2022-08-16 10:45:00', 0.07846703950321553]
P/L = 3.8093658173279366
107.02019611465832


['DOGE', '2022-08-17 22:15:00', 0.08279064351756689]
['DOGE', '2022-08-18 00:35:00', 0.08165]
P/L = -1.377744475829401
105.54573127466682


['DOGE', '2022-08-18 07:20:00', 0.08096746291185636]
['DOGE', '2022-08-18 07:25:00', 0.07957]
P/L = -1.725956157694697
103.72405822654781


['DOGE', '2022-08-19 09:15:00', 0.07634098110323118]
['DOGE', '2022-08-19 09:15:00', 0.07386]
P/L = -3.249867983588395
100.35316326696464


['DOGE', '2022-08-19 16:35:00', 0.07286734768571591]
['DOGE', '2022-08-19 16:35:00', 0.07032]
P/L = -3.495869915154426
96.844

In [34]:
a = [1,6,5,3]
x = 8

b = len(a) - 1
while b > 0:
    a[b] = a[b-1]
    b -= 1
a[0] = x


    


In [35]:
a

[8, 1, 6, 5]

In [10]:
print(races)


[]
